# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 03**: Data Cleaning and Transformation Pipeline

**Date**: September 18th 2025

**Student Name**: Juan Bernardo Orozco Quirarte

**Professor**: Pablo Camarillo Ramirez

### Find Spark Installation

In [1]:
import findspark
findspark.init()

### Create Spark Session

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on data sources (Files)") \
    .master("spark://7d7f1cd1d5d2:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/20 17:54:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Define Schema

In [3]:
from lib.bernardoorozco.spark_utils import SparkUtils
airlines_schema_columns = [("index", "int"), 
     ("airline", "string"), 
     ("flight", "string"),
     ("source_city", "string"),
     ("departure_time", "string"),
     ("stops", "string"),
     ("arrival_time", "string"),
     ("destination_city", "string"),
     ("class", "string"),
     ("duration", "float"),
     ("days_left", "int"),
     ("price", "int")
     ]
airlines_schema = SparkUtils.generate_schema(airlines_schema_columns)
airlines_schema

StructType([StructField('index', IntegerType(), True), StructField('airline', StringType(), True), StructField('flight', StringType(), True), StructField('source_city', StringType(), True), StructField('departure_time', StringType(), True), StructField('stops', StringType(), True), StructField('arrival_time', StringType(), True), StructField('destination_city', StringType(), True), StructField('class', StringType(), True), StructField('duration', FloatType(), True), StructField('days_left', IntegerType(), True), StructField('price', IntegerType(), True)])

### Load CSV

In [4]:
df_airlines = spark.read \
                .option("header", "true") \
                .schema(airlines_schema) \
                .csv("/opt/spark/work-dir/data/archive/airlines_flights_data.csv")

df_airlines.show(n=5)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|    3| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|    4| Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
+-----+--------+-------+

### Drop unnecessary columns. Count how many null values the dataset has before/after the cleaning process.

In [5]:
from pyspark.sql.functions import trim, col, count, isnull, when
print(f"number of records before cleaning: {df_airlines.count()}")
df_airlines.select([count(when(col(c).isNull(), c)).alias(c) for c in df_airlines.columns]).show()


number of records before cleaning: 300153


+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



In [6]:
df_clean = df_airlines.drop("index", "flight", "class", "days_left")    # We are not going to use this columns

for c in df_clean.columns:
    if dict(df_clean.dtypes)[c] == "string":
        df_clean = df_clean.withColumn(c, trim(col(c)))   # Trim of string columns

df_clean=df_clean.dropna()  # Drop null columns if exists

print(f"Numer of records after cleaning: {df_clean.count()}")
df_clean.select([count(when(col(c).isNull(), c)).alias(c) for c in df_clean.columns]).show()

Numer of records after cleaning: 300153


+-------+-----------+--------------+-----+------------+----------------+--------+-----+
|airline|source_city|departure_time|stops|arrival_time|destination_city|duration|price|
+-------+-----------+--------------+-----+------------+----------------+--------+-----+
|      0|          0|             0|    0|           0|               0|       0|    0|
+-------+-----------+--------------+-----+------------+----------------+--------+-----+



### Normalize categorical values: map “zero” → 0, “one” → 1, etc. in stops.

In [7]:
df_clean.select("stops").distinct().show()  # Not normalized

+-----------+
|      stops|
+-----------+
|two_or_more|
|        one|
|       zero|
+-----------+



In [8]:
df_clean=df_clean.withColumn("stops", when(col("stops") == "zero", 0).when(col("stops") == "one", 1).when(col("stops") == "two_or_more", 2))
df_clean.select("stops").distinct().show()  # Normalized

+-----+
|stops|
+-----+
|    1|
|    2|
|    0|
+-----+



### Create a new column called route: “Delhi → Mumbai” from source_city and destination_city.

In [9]:
from pyspark.sql.functions import concat_ws
df_clean = df_clean.withColumn("route", concat_ws(" -> ", col("source_city"), col("destination_city")))
df_clean.show(5)

+--------+-----------+--------------+-----+-------------+----------------+--------+-----+---------------+
| airline|source_city|departure_time|stops| arrival_time|destination_city|duration|price|          route|
+--------+-----------+--------------+-----+-------------+----------------+--------+-----+---------------+
|SpiceJet|      Delhi|       Evening|    0|        Night|          Mumbai|    2.17| 5953|Delhi -> Mumbai|
|SpiceJet|      Delhi| Early_Morning|    0|      Morning|          Mumbai|    2.33| 5953|Delhi -> Mumbai|
| AirAsia|      Delhi| Early_Morning|    0|Early_Morning|          Mumbai|    2.17| 5956|Delhi -> Mumbai|
| Vistara|      Delhi|       Morning|    0|    Afternoon|          Mumbai|    2.25| 5955|Delhi -> Mumbai|
| Vistara|      Delhi|       Morning|    0|      Morning|          Mumbai|    2.33| 5955|Delhi -> Mumbai|
+--------+-----------+--------------+-----+-------------+----------------+--------+-----+---------------+
only showing top 5 rows


### Transform departure_time and arrival_time to numerical categories (Morning, Afternoon, etc.), then encode as numbers (0=Early_Morning, 1=Morning, etc.).

In [10]:
df_clean.select("departure_time").distinct().show()
df_clean.select("arrival_time").distinct().show()

+--------------+
|departure_time|
+--------------+
|       Evening|
|       Morning|
|    Late_Night|
|     Afternoon|
| Early_Morning|
|         Night|
+--------------+



+-------------+
| arrival_time|
+-------------+
|      Evening|
|      Morning|
|   Late_Night|
|    Afternoon|
|Early_Morning|
|        Night|
+-------------+



In [11]:
df_clean = df_clean.withColumn(
    "departure_time",
    when(col("departure_time") == "Early_Morning", 0).when(col("departure_time") == "Morning", 1)
    .when(col("departure_time") == "Afternoon", 2).when(col("departure_time") == "Evening", 3)
    .when(col("departure_time") == "Night", 4).when(col("departure_time") == "Late_Night", 5)
)

df_clean = df_clean.withColumn(
    "arrival_time", when(col("arrival_time") == "Early_Morning", 0) .when(col("arrival_time") == "Morning", 1)
    .when(col("arrival_time") == "Afternoon", 2).when(col("arrival_time") == "Evening", 3)
    .when(col("arrival_time") == "Night", 4).when(col("arrival_time") == "Late_Night", 5)
)

In [12]:
df_clean.select("departure_time").distinct().show()
df_clean.select("arrival_time").distinct().show()

+--------------+
|departure_time|
+--------------+
|             1|
|             3|
|             5|
|             4|
|             2|
|             0|
+--------------+



+------------+
|arrival_time|
+------------+
|           1|
|           3|
|           5|
|           4|
|           2|
|           0|
+------------+



### Add a new column is_expensive: when(price > 6000, True).otherwise(False).

In [13]:
df_clean=df_clean.withColumn("is_expensive", when(col("price")>6000, True).otherwise(False))
df_clean.show(5)

+--------+-----------+--------------+-----+------------+----------------+--------+-----+---------------+------------+
| airline|source_city|departure_time|stops|arrival_time|destination_city|duration|price|          route|is_expensive|
+--------+-----------+--------------+-----+------------+----------------+--------+-----+---------------+------------+
|SpiceJet|      Delhi|             3|    0|           4|          Mumbai|    2.17| 5953|Delhi -> Mumbai|       false|
|SpiceJet|      Delhi|             0|    0|           1|          Mumbai|    2.33| 5953|Delhi -> Mumbai|       false|
| AirAsia|      Delhi|             0|    0|           0|          Mumbai|    2.17| 5956|Delhi -> Mumbai|       false|
| Vistara|      Delhi|             1|    0|           2|          Mumbai|    2.25| 5955|Delhi -> Mumbai|       false|
| Vistara|      Delhi|             1|    0|           1|          Mumbai|    2.33| 5955|Delhi -> Mumbai|       false|
+--------+-----------+--------------+-----+------------+

### Get the average price per airline.

In [14]:
from pyspark.sql.functions import avg
df_clean.groupBy("airline").agg(avg("price").alias("avg_price")).show()

+---------+------------------+
|  airline|         avg_price|
+---------+------------------+
|   Indigo| 5324.216303339517|
| SpiceJet| 6179.278881367218|
|Air_India| 23507.01911190229|
|  AirAsia|4091.0727419555224|
| GO_FIRST| 5652.007595045959|
|  Vistara| 30396.53630170735|
+---------+------------------+



### Average duration per route.

In [15]:
df_clean.groupBy("route").agg(avg("duration").alias("avg_duration")).show()

+--------------------+------------------+
|               route|      avg_duration|
+--------------------+------------------+
|    Delhi -> Chennai|12.433964745763944|
|  Hyderabad -> Delhi|10.829816602522587|
|   Mumbai -> Chennai|12.665900287564627|
|Hyderabad -> Kolkata|13.535322410033165|
| Hyderabad -> Mumbai|11.962923295795918|
| Mumbai -> Bangalore|11.612022516178817|
|    Delhi -> Kolkata| 12.73596614766045|
|   Mumbai -> Kolkata|12.836848115489666|
|Bangalore -> Kolkata|13.099143404859825|
| Mumbai -> Hyderabad|13.263310412247066|
|    Kolkata -> Delhi| 11.60498857561711|
|Hyderabad -> Chennai|13.293238468912078|
|     Delhi -> Mumbai|10.367774213738123|
|   Kolkata -> Mumbai|12.991932481150478|
|     Mumbai -> Delhi|  9.81805726844943|
|Kolkata -> Hyderabad|13.853107514948396|
|Bangalore -> Chennai|14.480207509137166|
|  Bangalore -> Delhi|  9.77995566082195|
|Bangalore -> Hyde...|14.162432783513621|
|Hyderabad -> Bang...| 12.09331678643705|
+--------------------+------------

### Minimum and maximum price per airline.

In [16]:
from pyspark.sql.functions import min, max
df_clean.groupBy("airline").agg(min("price").alias("min_price"), max("price").alias("max_price")).show()

+---------+---------+---------+
|  airline|min_price|max_price|
+---------+---------+---------+
|   Indigo|     1105|    31952|
| SpiceJet|     1106|    34158|
|Air_India|     1526|    90970|
|  AirAsia|     1105|    31917|
| GO_FIRST|     1105|    32803|
|  Vistara|     1714|   123071|
+---------+---------+---------+



### Count flights by departure_time category.

In [17]:
df_clean.groupBy("departure_time").count().show()

+--------------+-----+
|departure_time|count|
+--------------+-----+
|             1|71146|
|             3|65102|
|             5| 1306|
|             4|48015|
|             2|47794|
|             0|66790|
+--------------+-----+



In [18]:
sc.stop()